In [663]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geojson import GeometryCollection
import geojson

In [686]:
# Point data to geoJSON
import random
import sqlite3

df_red = pd.read_csv("redlist_assessments.csv")

# Assume: RedList scientific correspond to UBC species
red_sciNames = df_red.scientificName

def some(x, n):
    return x.loc[random.sample(x.index, n)]

def addCommonNames(geo_df):
    
    # df_commons['common'] = df_commons[['language', 'vernacular_name']].apply(lambda x: ':'.join(x), axis=1)
    # df_commons = df_commons.groupby(['tsn'])['common'].apply(', '.join).reset_index()
    db = sqlite3.connect('ITIS.sqlite')
    df_commons = pd.read_sql_query("SELECT * from vernaculars", db)
    df_species = pd.read_sql_query("SELECT * from longnames", db)

    df_commons = df_commons[df_commons['language'] == "English"]
    df_commons = df_commons.rename({'vernacular_name': 'common'}, axis=1)
    df_commons = df_commons[['tsn', 'common']]
    
    df_species = df_species[df_species['completename'].isin(geo_df.species)]
    df_commons = df_commons[df_commons['tsn'].isin(df_species.tsn)]
    
    df_commons = df_commons.groupby(['tsn'])['common'].apply(', '.join).reset_index()
    df_species = df_species[df_species['tsn'].isin(df_commons.tsn)]

    df_common_species = df_species.merge(df_commons, on="tsn")
    df_common_species = df_common_species.rename({'completename': 'species'}, axis=1) 
    df_common_species = df_common_species.drop(['tsn'], 1)

    species = set(geo_df.species.unique())
    species_in_itis = set(df_common_species.species.unique())
    diff = list(species - species_in_itis)
    diff = {df_common_species.columns[0]: diff, df_common_species.columns[1]: 'unkown'}
    df = pd.DataFrame.from_dict(diff)
    df_species = pd.concat([df_common_species, df], sort = False)

    geo_df = geo_df.merge(df_species, on="species")
    return geo_df

def redList(x, red_sciNames):
    if (x['species'] in red_sciNames.values):
        return df_red.loc[red_sciNames[red_sciNames == x['species']].index[0]].redlistCategory
    else:
        return float('nan')

def pointCSVtoJSON(filename, num_rows=0):
    df = pd.read_csv(filename, delimiter="\t", low_memory=False)
    if (num_rows):
        df = some(df, num_rows)
    geometry = [Point(xy) for xy in zip (df['decimalLongitude'], df['decimalLatitude'])]
    geo_df = gpd.GeoDataFrame(df, geometry = geometry, crs = {'init': 'epsg:4326'})
    
    # gbif_geo_df = gbif_geo_df[gbif_geo_df['species'].isin(red_sciNames)]
#     geo_df['redList'] = geo_df.apply(lambda x: int(x['species']in red_sciNames.values), 1).values
    geo_df['redList'] = geo_df.apply(lambda x: redList(x, red_sciNames), 1).values
    
    
    # Common Names ITIS
    # # Get names of all tables
    # c = conn.cursor()
    # c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # print(c.fetchall())
    
    geo_df = addCommonNames(geo_df)
    geo_df.to_file("leaflet/" + filename.split('.')[0] + '.geojson', driver="GeoJSON")
    
    return geo_df

In [688]:
geo_df = pointCSVtoJSON('gbif_tot.csv', 2000)

In [700]:
df = pd.read_csv('gbif_tot.csv', delimiter="\t", low_memory=False)

In [712]:
df[df['institutionCode'].isnull()]

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
40477,2234880896,ad43e954-dd79-4986-ae34-9ccdbd8bf568,MG305344,Animalia,Arthropoda,Insecta,Diptera,Chironomidae,Gymnometriocnemus,NaN,...,Kate Perez,NaN,CC_BY_4_0,NaN,J.Sibbald,NaN,NaN,2019-04-05T04:28:50.291Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;COUNTRY_DERIVED_F...
40478,2234880917,ad43e954-dd79-4986-ae34-9ccdbd8bf568,MG305664,Animalia,Arthropoda,Insecta,Diptera,Chironomidae,NaN,NaN,...,Kate Perez,NaN,CC_BY_4_0,NaN,J.Sibbald,NaN,NaN,2019-04-05T04:28:50.367Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;COUNTRY_DERIVED_F...
40479,2234880952,ad43e954-dd79-4986-ae34-9ccdbd8bf568,MG302645,Animalia,Arthropoda,Insecta,Diptera,Chironomidae,Smittia,NaN,...,Kate Perez,NaN,CC_BY_4_0,NaN,Rosemary Venos,NaN,NaN,2019-04-05T04:28:50.581Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;COUNTRY_DERIVED_F...
40480,2234880977,ad43e954-dd79-4986-ae34-9ccdbd8bf568,MG303510,Animalia,Arthropoda,Insecta,Diptera,Chironomidae,Gymnometriocnemus,NaN,...,Kate Perez,NaN,CC_BY_4_0,NaN,J.Sibbald,NaN,NaN,2019-04-05T04:28:50.649Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;COUNTRY_DERIVED_F...
40481,2234880992,ad43e954-dd79-4986-ae34-9ccdbd8bf568,MG302866,Animalia,Arthropoda,Insecta,Diptera,Chironomidae,Gymnometriocnemus,NaN,...,Kate Perez,NaN,CC_BY_4_0,NaN,J.Sibbald,NaN,NaN,2019-04-05T04:28:50.699Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;COUNTRY_DERIVED_F...
40482,2234881016,ad43e954-dd79-4986-ae34-9ccdbd8bf568,MG300138,Animalia,Arthropoda,Insecta,Diptera,Chironomidae,Gymnometriocnemus,NaN,...,Kate Perez,NaN,CC_BY_4_0,NaN,J.Sibbald,NaN,NaN,2019-04-05T04:28:50.834Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;COUNTRY_DERIVED_F...
40483,2234881019,ad43e954-dd79-4986-ae34-9ccdbd8bf568,MG301989,Animalia,Arthropoda,Insecta,Diptera,Chironomidae,NaN,NaN,...,Kate Perez,NaN,CC_BY_4_0,NaN,J.Sibbald,NaN,NaN,2019-04-05T04:28:50.858Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;COUNTRY_DERIVED_F...
40484,2234881022,ad43e954-dd79-4986-ae34-9ccdbd8bf568,MG300407,Animalia,Arthropoda,Insecta,Diptera,Chironomidae,NaN,NaN,...,Kate Perez,NaN,CC_BY_4_0,NaN,J.Sibbald,NaN,NaN,2019-04-05T04:28:50.886Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;COUNTRY_DERIVED_F...
40485,2234881045,ad43e954-dd79-4986-ae34-9ccdbd8bf568,MG305401,Animalia,Arthropoda,Insecta,Diptera,Chironomidae,Gymnometriocnemus,NaN,...,Kate Perez,NaN,CC_BY_4_0,NaN,J.Sibbald,NaN,NaN,2019-04-05T04:28:50.997Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;COUNTRY_DERIVED_F...
40486,2234881054,ad43e954-dd79-4986-ae34-9ccdbd8bf568,MG305600,Animalia,Arthropoda,Insecta,Diptera,Chironomidae,NaN,NaN,...,Kate Perez,NaN,CC_BY_4_0,NaN,J.Sibbald,NaN,NaN,2019-04-05T04:28:51.058Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;COUNTRY_DERIVED_F...


In [697]:
geo_df.columns

Index([u'gbifID', u'datasetKey', u'occurrenceID', u'kingdom', u'phylum',
       u'class', u'order', u'family', u'genus', u'species',
       u'infraspecificEpithet', u'taxonRank', u'scientificName',
       u'countryCode', u'locality', u'publishingOrgKey', u'decimalLatitude',
       u'decimalLongitude', u'coordinateUncertaintyInMeters',
       u'coordinatePrecision', u'elevation', u'elevationAccuracy', u'depth',
       u'depthAccuracy', u'eventDate', u'day', u'month', u'year', u'taxonKey',
       u'speciesKey', u'basisOfRecord', u'institutionCode', u'collectionCode',
       u'catalogNumber', u'recordNumber', u'identifiedBy', u'dateIdentified',
       u'license', u'rightsHolder', u'recordedBy', u'typeStatus',
       u'establishmentMeans', u'lastInterpreted', u'mediaType', u'issue',
       u'geometry', u'redList', u'common'],
      dtype='object')

In [307]:
# SHP data to geoJSON
def fix_crs(map_ob):
    return map_ob.to_crs({'init': 'epsg:4326'})
str_map = gpd.read_file("ecological_reserves/BC_Eco_Reserves.shp")
str_map = fix_crs(str_map)

In [308]:
# # GeoJSON does not support multipolygon. Doesn't work
# str_map.to_file("leaflet/UBC_poly.geojson", driver="GeoJSON")

# # fiona doesn't work
# import fiona
# import json

# with fiona.open('ecological_reserves/BC_Eco_Reserves.shp') as source:
#     records = list(source)
# geo_json = {"type": "FeatureCollection","features": records}
# with open('leaflet/UBC_poly.geojson', 'w') as fp:
#     json.dump(geo_json, fp)

In [309]:
# Convert multipolygon to single polygons

In [310]:
geom_series = str_map.geometry

In [311]:
def geom_apply(x):
    try:
        return list(x)
    except TypeError:
        return [x]

In [312]:
geom_series = geom_series.apply(geom_apply)

In [313]:
geom_series = geom_series.apply(pd.Series).stack()

In [314]:
# # Quick but loses properties
# list_poly = list(geom_series)
# geoms = GeometryCollection(list_poly)
# geo_file = geojson.dumps(geoms)
# with open("leaflet/UBC_poly.geojson", "w") as text_file:
#     text_file.write(geo_file)

In [315]:
# Careful! Deep copy required here to avoid chaining

df = pd.DataFrame(columns=str_map.columns)
for ind, poly in geom_series.iteritems():
    curr_row = str_map.loc[ind[0]].copy(deep=True)
    curr_row['geometry'] = poly
    df = df.append(curr_row)

df_gpd = gpd.GeoDataFrame(df,geometry = df.geometry, crs = {'init': 'epsg:4326'})
df_gpd.to_file("leaflet/UBC_poly.geojson", driver="GeoJSON")